In [ ]:
# manually mark the boundaries

import numpy as np
import glob
from PIL import Image
import matplotlib
# using the macosx backend, but this might not be appropriate for non-mac users.
# alternatively try TkAgg or check this site:
# https://matplotlib.org/faq/usage_faq.html#what-is-a-backend
#matplotlib.use('macosx')
import matplotlib.pyplot as plt
import pickle
from sklearn.cluster import KMeans
from skimage.measure import grid_points_in_poly
from scipy.interpolate import splprep, splrep, splev
from scipy.ndimage.measurements import center_of_mass
from scipy.optimize import curve_fit
from scipy.ndimage.interpolation import zoom
from natsort import natsorted, ns
import matplotlib.pyplot as plt

# a list of folder names where the image stacks are located.
buds = ['sample17_batch3_IJ_stack_ok']

for d in buds:
    files = []
    for file in glob.glob('image_stacks/'+d+'/*.ti*'):
        files.append(file)
    #  natural sort the file names (necessary if the numbers in the file names are not padded with zeros)
    files = natsorted(files)
    files = np.array(files)
    
    # collect 40 frames to label
    frames = np.linspace(0,len(files),num=40,endpoint=False).astype(int)
    print(files[frames])

    # loop through these files, show the images, and the trained user needs to click around the shell-leaf boundary
    shell_leaf = []
    for f in frames:
       	im = np.array(Image.open(files[f]))

        # show image and click around the shell-leaf boundary 
        # click as many times as you'd like. the first and last clicks will be connected to close the boundary
        # press enter to terminate clicking
        plt.title('Click on the shell-leaf boundary')
        plt.axis('equal')
        plt.xlim([0,np.shape(im)[0]])
        plt.ylim([0,np.shape(im)[1]])
        plt.imshow(im.T,cmap='Greys_r')
        coord = plt.ginput(n=-1,timeout=-1)
        plt.close()
        shell_leaf.append(coord)

    # save the clicks and the clicked file names
    file_name = 'data_files/'+d+'_shell_leaf_boundaries.dat'
    f = open(file_name,'wb')
    pickle.dump([shell_leaf,files[frames]],f)
    f.close()




In [1]:
# segment and calculate features
import numpy as np
import glob
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib
import pickle
from sklearn.cluster import KMeans
from skimage.measure import grid_points_in_poly
from scipy.interpolate import splprep, splrep, splev
from scipy.ndimage.measurements import center_of_mass
from scipy.optimize import curve_fit
from scipy.ndimage.interpolation import zoom
import sys
sys.path.insert(0, 'pydescriptors/pydescriptors/')
from compactness import hz as compactness_hz
import nibabel as nib
from natsort import natsorted, ns
import pandas as pd

def func(x, C, D): 
    return C*x**D

buds = ['sample17_batch3_IJ_stack_ok']

# set it to true if you want the code to produce figures
plot = False

# the results will be stored in a pandas dataframe
df_results = pd.DataFrame()

for d in buds:
    # open image files
    print(d)
    print('   open and cluster images...')

    files = []
    for file in glob.glob('image_stacks/'+d+'/*.ti*'):
        files.append(file)

    files = natsorted(files)

    scale = False    
    im = np.array(Image.open(files[0]))
    if np.any(np.array([len(files),np.shape(im)[0],np.shape(im)[1]])>1000):
        print('   DOWNSCALING imagestack by a factor of 2!')
        scale = True
    
    if scale:
        im = zoom(im,0.5)
        image_stack = np.zeros([int(len(files)/2+1),np.shape(im)[0],np.shape(im)[1]])
        for i in range(len(files)):
            if i%2 == 0:
                im = zoom(np.array(Image.open(files[i])),0.5)
                image_stack[int(i/2),:,:] = im
    else:
        image_stack = np.zeros([len(files),np.shape(im)[0],np.shape(im)[1]])
        for i in range(len(files)):
            im = np.array(Image.open(files[i]))
            image_stack[i,:,:] = im

    # do the clustering on the whole image stack 
    cls = KMeans(2).fit_predict(image_stack.reshape([np.prod(np.shape(image_stack)),1]))            
    # make sure the background is 0, the bud is 1, clustering randomizes which one is which
    # assuming here that the corner pixel is always background
    if cls[0] != 0:
        cls ^= 1
    bud_voxels = cls.reshape(np.shape(image_stack))

    print(np.shape(image_stack),np.shape(bud_voxels))
    print(len(files),np.shape(im)[0],np.shape(im)[1])
    print('   done')

    # find all leaf voxels
    leaf_voxels = np.zeros(np.shape(image_stack))

    print('   find leaf voxels...')

    # load the clicks
    file_name = 'data_files/'+d+'_shell_leaf_boundaries_d1.dat'
    print(file_name)
    
    # open clicks and the file names of clicked images
    f = open(file_name,'rb')
    [shell_leaf,fs] = pickle.load(f)
    f.close()
      
    indx = np.array([i for i, ltr in enumerate(str(fs[0])) if ltr == '.'])
    frames = np.array([int(str(f)[indx[-1]-4:indx[-1]]) for f in fs])
    
    if scale:
        frames = frames/2
    print(frames)
    # do the periodic spline interpolation in the x-y frame and collect 1000 points on each frame
    points = np.zeros([len(frames),3,1000])
    for f in range(len(frames)):

        boundary = np.array(shell_leaf[f])

        if len(boundary)>3:
            if scale:
                boundary = boundary / 2e0
                
            center = [np.sum(boundary[:,0])/len(boundary[:,0]),np.sum(boundary[:,1])/len(boundary[:,1])]

            tck, u = splprep(boundary.T, u=None, s=0.0, per=1) 

            u_new = np.linspace(u.min(), u.max(), 1000)
            x_new, y_new = splev(u_new, tck, der=0)

            phi = np.arctan2(y_new-center[1], x_new-center[0])/np.pi
            indx = int(np.where(phi == np.max(phi))[0])

            x_shift = np.append(x_new[indx:],x_new[:indx])
            y_shift = np.append(y_new[indx:],y_new[:indx])
            z_shift = np.zeros(len(x_new))+frames[f]

            points[f] = np.array([x_shift,y_shift,z_shift])    

    # do the vertical interpolation and collect the tcks for splev
    splev_x = []
    splev_y = []
    for i in range(1000):
        x = points[:,0,i]
        y = points[:,1,i]
        z = points[:,2,i]
        tckx = splrep(z[x>0],x[x>0])
        splev_x.append(tckx)

        tcky = splrep(z[x>0],y[x>0])
        splev_y.append(tcky)

    # go through the frames and plot the interpolated contours

    alpha = 0.4
    colors = [(0.0, 0.0, 0.0, 0e0),  # black
                (1.0, 0.0, 0.0, alpha)]  # red
    cmap1 = matplotlib.colors.ListedColormap(colors)
    colors = [(0.0, 0.0, 0.0, 0e0),  # black
                (0.0, 0.0, 1.0, alpha)]  # blue
    cmap2 = matplotlib.colors.ListedColormap(colors)
    
    # check in between which frames should the leaf voxels be found
    length = np.zeros(len(frames))
    for f in range(len(frames)):
        length[f] = len(np.array(shell_leaf[f]))

    i_min = frames[np.min(np.where(length>0)[0])]
    i_max = frames[np.max(np.where(length>0)[0])]
    
    boundary_surface = np.zeros(np.shape(bud_voxels))
    
    f = 0
    for i in range(np.shape(image_stack)[0]):
        if plot:
            plt.figure(figsize=(6,6))
            plt.title('blue: scale, red: leaf, uncolored: background')
            plt.xlabel('pixel coord.')
            plt.ylabel('pixel coord.')
            plt.imshow(image_stack[i,:,:].T,cmap='Greys_r',vmin=10000,vmax=33000)
        if (i == frames[f+1]) and (f < len(frames)-2):
            f = f + 1
        boundary = np.array(shell_leaf[f])
        if (i >= i_min) and (i <= i_max):
            if scale:
                boundary = boundary / 2e0
            x_cont = np.zeros(1000)
            y_cont = np.zeros(1000)
            for j in range(1000):
                tck_x = splev_x[j]
                tck_y = splev_y[j]
                x_cont[j] = splev(i,tck_x)
                y_cont[j] = splev(i,tck_y)
            pixels_in_cont = grid_points_in_poly(np.shape(bud_voxels[i,:,:]),np.array([x_cont,y_cont]).T)
            pixels_in_cont = np.rint(pixels_in_cont == True)
            leaf_voxels[i,:,:] = bud_voxels[i,:,:]*pixels_in_cont
            boundary_surface[i,:,:] = pixels_in_cont
            if plot: 
                plt.imshow(leaf_voxels[i,:,:].T,cmap=cmap1)
                plt.imshow((bud_voxels[i,:,:]-leaf_voxels[i,:,:]).T,cmap=cmap2)
                plt.contour(pixels_in_cont.T,levels=[0.5],colors=['y'])
                if i == frames[f]-1:
                    plt.plot(boundary[:,0],boundary[:,1],'wo')
                if i == frames[f]:
                    plt.plot(boundary[:,0],boundary[:,1],'wo')
                if i == frames[f]+1:
                    plt.plot(boundary[:,0],boundary[:,1],'wo')
        else:
            if plot:
                plt.imshow(bud_voxels[i,:,:].T,cmap=cmap2)

        if plot:
            plt.xlim([0,np.shape(image_stack[i,:,:])[0]])
            plt.ylim([0,np.shape(image_stack[i,:,:])[1]])    
            plt.savefig('animation/interpolate/frame'+str(i).zfill(4)+'.png',dpi=150)
            plt.close()
    

    if plot:
        # plot one vertical frame with the interpolated non-periodic boundaries
        coord = int(3*np.shape(boundary_surface)[2]/7)

        boundary = boundary_surface[:,:,coord]
        plant = image_stack[:,:,coord]
        leaf = leaf_voxels[:,:,coord]
        bud = bud_voxels[:,:,coord]
        plt.figure(figsize=(4,6))
        plt.title('blue: scale, red: leaf, uncolored: background')
        plt.xlabel('pixel coord.')
        plt.ylabel('pixel coord.')
        plt.imshow(plant,cmap='Greys_r',vmin=10000,vmax=33000,origin='lower')
        plt.imshow(leaf,cmap=cmap1,origin='lower')
        plt.imshow((bud-leaf),cmap=cmap2,origin='lower')
        cs = plt.contour(boundary,levels=[0.5],colors=['g'],alpha=0)

        points = cs.collections[0].get_paths()[0].vertices
        all_points = []

        for f in frames:
            indx = np.where(points[:,1] == f)[0]
            if len(indx) > 0:
                plt.plot([0,333],[f,f],'w',linewidth=0.5)
                #plt.plot(points[indx,0],points[indx,1],'wo')
                all_points.append(points[indx])

        all_points = np.array(all_points)

        tck1 = splrep(all_points[:,0,1],all_points[:,0,0])
        tck2 = splrep(all_points[:,1,1],all_points[:,1,0])
        z = np.arange(frames[28])
        plt.plot(splev(z,tck1,ext=1),z,'y',linewidth=3)
        plt.plot(splev(z,tck2,ext=1),z,'y',linewidth=3)

        plt.savefig('animation/vertical_frame_test.png',dpi=150)
        plt.show()
        plt.close()
        
        # illustrate fractal dimension in 2D
        CoM = center_of_mass(leaf)
        
        scale = 2e0

        nr_radii = int(np.max(np.shape(leaf_voxels))/scale)
        radii = np.arange(nr_radii+1)*scale

        x,y = np.ogrid[-CoM[0]:np.shape(leaf_voxels)[0]-CoM[0], -CoM[1]:np.shape(leaf_voxels)[1]-CoM[1]]
        count = np.zeros(len(radii))
        
        for i in range(nr_radii):
            mask_outer = x*x + y*y <= radii[i+1]**2e0
            count[i] = np.sum(leaf_voxels[mask_outer])
        
        count = 1e0*count/np.max(count)
        
        max_rad = np.argmax(radii[count < np.max(count)])
        [C,D], pcov = curve_fit(func, radii[1:int(max_rad/3)], count[1:int(max_rad/3)])
        

        plt.figure(figsize=(10,6))
        plt.subplot(1,2,1)
        
        plt.imshow(leaf,cmap='Greys_r',origin='lower')
        plt.xlabel('pixel coord.')
        plt.ylabel('pixel coord.')
        plt.title('white: leaf')
        plt.plot([CoM[0]],[CoM[1]],'or')
        for r in radii[:max_rad:10]:
            circle = plt.Circle(CoM,r,color='r', fill=False)
            plt.gcf().gca().add_artist(circle)
        circle = plt.Circle(CoM,radii[max_rad]/3,color='r', fill=False,linewidth=3)
        plt.gcf().gca().add_artist(circle)
        
        plt.subplot(1,2,2)

        plt.plot(radii[:max_rad],count[:max_rad],label='pixel count')
        plt.plot(radii[:int(max_rad/3)],count[:int(max_rad/3)],'r',linewidth=3,label='inner third pixel count')
        plt.plot(radii[:int(max_rad/2)],C*radii[:int(max_rad/2)]**D,'k--',label='c*r^D fit, D = '+str(np.round(D,2)))
        plt.xlabel('radius')
        plt.ylabel('normalized pixel count')
        plt.legend()
        plt.savefig('animation/fractalD_test.png',dpi=150)
        plt.show()
        plt.close()
        

    # pickle dump the arrays
    file_name = 'data_files/segmented_test_'+d+'.dat'
    f = open(file_name,'wb')
    pickle.dump([bud_voxels+leaf_voxels],f)
    f.close()

    # save images as nifti files to later process with ITK-SNAP
    nimg = nib.Nifti1Image(leaf_voxels, np.eye(4))
    nimg.get_data_dtype() == np.dtype(int)
    nib.save(nimg, 'data_files/segmented_leaf_test_'+d+'.nii.gz')
    
    nimg = nib.Nifti1Image(bud_voxels+leaf_voxels, np.eye(4))
    nimg.get_data_dtype() == np.dtype(int)
    nib.save(nimg, 'data_files/segmented_test_'+d+'.nii.gz')
    
    print('   done')

    # calculate features

    print('   calculating features...')

    # nr of bud voxels
    nr_bud_vox = np.sum(bud_voxels)

    # leaf to bud volume ratio
    lf_to_bud = np.sum(leaf_voxels)/nr_bud_vox

    # compactness
    X,Y,Z = leaf_voxels.nonzero()
    comp = compactness_hz(X,Y,Z)

    # fractal dimension and normalized counts
    CoM = center_of_mass(leaf_voxels)
    
    scale = 2e0

    nr_radii = int(np.max(np.shape(leaf_voxels))/scale)
    radii = np.arange(nr_radii+1)*scale
    x,y,z = np.ogrid[-CoM[0]:np.shape(leaf_voxels)[0]-CoM[0], -CoM[1]:np.shape(leaf_voxels)[1]-CoM[1],-CoM[2]:np.shape(leaf_voxels)[2]-CoM[2]]

    count = np.zeros(len(radii))

    for i in range(nr_radii):
        mask_outer = x*x + y*y + z*z <= radii[i+1]**2e0
        count[i] = np.sum(leaf_voxels[mask_outer])

    if np.max(count) != np.sum(leaf_voxels):
        print('problem with shell histo')
        print(np.max(count), np.sum(leaf_voxels))
        raise ValueError

    max_rad = np.argmax(radii[count < np.max(count)])

    # fit power law
    [C,D], pcov = curve_fit(func, radii[1:int(max_rad/3)], count[1:int(max_rad/3)])

    perr = np.sqrt(np.diag(pcov))
    
    file_name = 'data_files/shell_hist_test_'+d+'.dat'
    f = open(file_name,'wb')
    pickle.dump([count,radii],f)
    f.close()
    
    # collect leaf/bud properties
    # NOTE I: the whole bud is used to calculate the number of bud voxels and the leaf to bud voxel ratio.
    # This is not always accurate because some buds might have a stem.
    # If you want more accurate results, load the nifti files (generated above) into ITK-SNAP and segment there.
    # NOTE II: this code doesn't know the size of the voxels. If you want actual volumes, you need to modify the 
    # csv file and multiple the voxel counts with the volume of one voxel.
    columns = ['bud', 'leaf_voxels', 'bud_voxels', 'leaf_bud_ratio', 'compactness', 'fractal_dimension_D', \
               'sigma_D', '25_centerMass', '50_centerMass', '75_centerMass']
    values = [d,np.sum(leaf_voxels),nr_bud_vox,lf_to_bud,comp,D,perr[1],count[radii == radii[int(max_rad/4e0)-1]]/np.max(count),    count[radii == radii[int(max_rad/2e0)-1]]/np.max(count),count[radii == radii[int(3e0*max_rad/4e0)-1]]/np.max(count)]
    df_bud = pd.DataFrame([values],columns=columns)
    
    # append results to df_results
    df_results = df_results.append(df_bud,ignore_index=True)

# save file to csv
df_results.to_csv('data_files/results.csv',index=False)

    

ModuleNotFoundError: No module named 'pandas'

In [ ]:
# animate some buds and leaves

import os
import numpy as np
import nibabel as nib
import mayavi
from mayavi import mlab
import pickle


nim = nib.load('data_files/segmented_sample17_batch3_IJ_stack_ok.nii.gz').get_fdata()
print(np.shape(nim))
# load the clicks
file_name = 'data_files/sample17_batch3_IJ_stack_ok_shell_leaf_boundaries_d1.dat'
f = open(file_name,'rb')
[shell_leaf,fs] = pickle.load(f)
f.close()

indx = np.array([i for i, ltr in enumerate(fs[0]) if ltr == '.'])
frames = np.array([int(f[indx[-1]-4:indx[-1]]) for f in fs])
# collect the points
x = []
y = []
z = []

for f in range(len(frames)):

    boundary = np.array(shell_leaf[f])

    if len(boundary)>3:
        for i in range(len(boundary[:,0])):
            y.append(boundary[i,0])
            x.append(boundary[i,1])
            z.append(frames[f])
x = np.array(x)
y = np.array(y)
z = np.array(z)

print(np.unique(nim))

for i in np.arange(360)[:1]:
    # make the plot with mlab
    fig = mlab.figure(bgcolor=(1,1,1),size=(480,640))
    source = mlab.pipeline.scalar_field(nim)
    vol = mlab.pipeline.volume(source, vmin=0.75,vmax=20,color=(0.0,0.5,0.0))

    mlab.points3d(x/2,y/2,z/2,color=(1,0,0),scale_factor=5)

    mlab.view(azimuth = i,distance=1000e0)
    mlab.savefig('animation/leaf/leaf_test_sample17_batch3_IJ_stack_ok_'+str(i).zfill(3)+'.jpg')
    mlab.close()
    